In [23]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import importlib
import funcoes
importlib.reload(funcoes)
import best_modelo
from best_modelo import *
import importlib
importlib.reload(best_modelo)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'best_modelo' from '/home/matheus/Documents/Mentoria/Avaliação Final/scrip_train/best_modelo.py'>

In [24]:
df_train = pd.read_csv("train_binary_small.csv")

df_test = pd.read_csv("test_binary_small.csv")
df_test.head()

,id,comment_text,Toxic
0,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",Toxic
1,0013a435effa29bd,"I'd never think I'd need to say it, but Wikipe...",Non-Toxic
2,001eff4007dbb65b,"::No, he is an arrogant, self serving, immatur...",Toxic
3,008eb47c4684d190,""" \n\n So, on the tenth anniversary of 9/11, ""...",Toxic
4,00991697be5ed76a,Simple: You are stupid!,Toxic


In [26]:
link_and_users = funcoes.Link()
stop = funcoes.Stopword()
sub_re = funcoes.Sub_re()
one_word = funcoes.One_carct()
lem = funcoes.Lematization()
stem = funcoes.Stemming()

# Preparando o dataset de treino:

In [27]:
df_train['Toxic'] = np.where(df_train['Toxic'] == 'Toxic', 1, 0)
df_test['Toxic'] = np.where(df_test['Toxic'] == 'Toxic', 1, 0)
df_train.drop("id", axis=1, inplace=True)
df_train.drop_duplicates(["comment_text"], inplace = True)
df_train["limpeza_simples"] = df_train['comment_text'].apply(lambda x: re.sub(r'\W+',' ', x))
df_train["limpeza_simples"] = df_train["limpeza_simples"].apply(lambda x: link_and_users.re_links_and_users(x))
df_train["limpeza_simples"] = df_train["limpeza_simples"].str.lower()

df_train_copy = df_train.copy()
df_train_copy["limpeza_simples"] = df_train_copy['limpeza_simples'].apply(lambda x: sub_re.sub_re(x))
df_train_copy["limpeza_simples"] = df_train_copy['limpeza_simples'].apply(lambda x: one_word.words_one(x))


# Limpeza simples, stemming
df_train_copy["limpeza_simples_stem"] = df_train_copy['limpeza_simples'].apply(lambda x: stem.stemming(x))



# Preparando o Dataset de test:

In [28]:
df_test['Toxic'] = np.where(df_test['Toxic'] == 'Toxic', 1, 0)
df_test.drop("id", axis=1, inplace=True)
df_test.drop_duplicates(["comment_text"], inplace = True)
df_test["limpeza_simples"] = df_test['comment_text'].apply(lambda x: re.sub(r'\W+',' ', x))
df_test["limpeza_simples"] = df_test["limpeza_simples"].apply(lambda x: link_and_users.re_links_and_users(x))
df_test["limpeza_simples"] = df_test["limpeza_simples"].str.lower()

df_test_copy = df_test.copy()

df_test_copy["limpeza_simples"] = df_test_copy['limpeza_simples'].apply(lambda x: sub_re.sub_re(x))
df_test_copy["limpeza_simples"] = df_test_copy['limpeza_simples'].apply(lambda x: one_word.words_one(x))

# Limpeza simples, stemming
df_test_copy["limpeza_simples_stem"] = df_test_copy['limpeza_simples'].apply(lambda x: stem.stemming(x))


In [31]:
list_preprocessing =[
    'limpeza_simples',
    'limpeza_simples_stem',
    ]

In [32]:
cv = 2
all_scores = {}
model_params = {}
vectorizer_name = 'BOW'

vectorizer = CountVectorizer()
model = MultinomialNB()

pipeline = Pipeline([("vect", vectorizer), ("clf", model)])

for param_name, parameters in VECTORIZERS_PARAMS.items():
        new_name = 'vect__'+param_name
        model_params[new_name] = parameters

for step in list_preprocessing:
    X_train = df_train_copy[step].values.astype('U')
    y_train = df_train_copy['Toxic'].to_list()

    X_test = df_test_copy[step].values.astype('U')
    y_test = df_test_copy['Toxic'].to_list()

    approach_name = (f"{step}__{'Naive_Bayes'}__{vectorizer_name}")
    
    rs = RandomizedSearchCV(
            pipeline, model_params, random_state=42, cv=cv, n_jobs=2,)

    rs.fit(X_train, y_train)
    
    print(f'[INFO] {approach_name}')
    print(rs.best_score_)

    dst_file = f'/home/matheus/Documents/Mentoria/Avaliação Final/scrip_train/list_params/{approach_name}.pkl'
    joblib.dump(rs,dst_file)

    y_pred = rs.predict(X_test)

    scores = {
        'accuracy':accuracy_score(y_test, y_pred),
        'f1':f1_score(y_test, y_pred),
        'confusion_matrix':confusion_matrix(y_test, y_pred),
        'best_param':rs.best_params_,
    }

    all_scores.update({approach_name:{'scores':scores}})

[INFO] limpeza_simples__Naive_Bayes__BOW
0.856
[INFO] limpeza_simples_stem__Naive_Bayes__BOW
0.8634999999999999
